# About
In this notebook we'll be building an Alpaca-based dataset containing every comment scraped from r/Hasan_Piker, specifically prepared for finetuning Llama3.1.

Theoretically, this should be able to turn into a script, where attributes are flags

WIP

## 1.0 - Preparation

### 1.1 - Import Packages

In [311]:
# import libs
import json
import pandas as pd
from loguru import logger

### 1.2 - Set Configuration Variables
Change these to build different datasets

In [ ]:
SUBREDDIT = "Destiny"
CLEANED_DATA = '../../json/output/cleaned_data_DEVELOPMENT.json'

class AlpacaPrompt:
    def __init__(self, instruction: str, input: str, response: str):
        self.instruction = instruction
        self.input = input
        self.response = response

    def to_dict(self):
        return {
            "instruction": self.instruction,
            "input": self.input,
            "response": self.response
        }

## 2.0 - Filter & Transform

### 2.1 - Load Cleaned Data

In [313]:
# load data into pandas dataframe
with open(CLEANED_DATA, 'r') as f:
  data = json.load(f)

df = pd.DataFrame(data)

# logging
prefilter_len = len(df)
logger.info(f"Data loaded: {df.shape}")
df.head(1)

2024-11-24 22:50:22.037 | INFO     | __main__:<module>:9 - Data loaded: (46, 25)


,id,subreddit,selftext,title,downs,name,upvote_ratio,ups,removed_by_category,link_flair_text,...,no_follow,created_utc,author_flair_text,author,num_comments,subreddit_subscribers,send_replies,is_video,deleted,comments
0,1dx1b0z,Destiny,,New Vegan,0,t3_1dx1b0z,0.95,121,None,Shitpost,...,False,1720304607,None,TuningsGaming,2,248289,True,False,False,"[{'id': 'lbyv8mn', 'total_awards_received': 0,..."


### 2.2 - Drop Unwanted Subreddits

In [314]:
# only keep the rows where the subreddit is in SUBREDDITS
df = df[df['subreddit'] == SUBREDDIT]

# logging
posts_removed = prefilter_len - len(df)
logger.info(f"Data filtered: {df.shape}")
logger.info(f"{posts_removed} post{'s' if posts_removed > 1 else ''} removed")
df.head(1)

2024-11-24 22:50:22.053 | INFO     | __main__:<module>:6 - Data filtered: (45, 25)
2024-11-24 22:50:22.055 | INFO     | __main__:<module>:7 - 1 post removed


,id,subreddit,selftext,title,downs,name,upvote_ratio,ups,removed_by_category,link_flair_text,...,no_follow,created_utc,author_flair_text,author,num_comments,subreddit_subscribers,send_replies,is_video,deleted,comments
0,1dx1b0z,Destiny,,New Vegan,0,t3_1dx1b0z,0.95,121,None,Shitpost,...,False,1720304607,None,TuningsGaming,2,248289,True,False,False,"[{'id': 'lbyv8mn', 'total_awards_received': 0,..."


## 3.0 - Bucket Data by Instruction Type

### 3.1 - Define instructions

In [315]:
instruction_types = [
  { 
    "instruction": f"Write a response to this post from r/{SUBREDDIT}",
    "response": []
  },
  { 
    "instruction": f"Write a response to this comment from r/{SUBREDDIT}",
    "responses": []
  }
]

# TODO: MAKE INSTRUCTIONS MORE SPECIFIC LIKE:
# instructions = [
#   { 
#     "instruction": f"Write a controverial response to this comment from r/{SUBREDDIT}",
#     "response": []
#   },
#   { 
#     "instruction": f"Write a likely upvoted response to this comment from r/{SUBREDDIT}",
#     "responses": []
#   },
#   { 
#     "instruction": f"Write a likely downvoted response to this comment from r/{SUBREDDIT}",
#     "responses": []
#   },
#   { 
#     "instruction": f"Write a response that is likely to be removed by the moderators to this comment from r/{SUBREDDIT}",
#     "responses": []
#   }
# ]

## 3.2 - ROUGH DRAFT

In [316]:
def is_comment_valid(comment) -> tuple[bool, str]:
  if (comment.startswith('!') and comment != "!") or 'http:' in comment or 'https:' in comment:
    return False, f"Comment is a bot call or contains a link: {comment}"
  match comment:
    case '[deleted]':
      return False, "Comment was deleted"
    case '[removed]':
      return False, "Comment was removed"
    case _:
      return True, ""
    
def is_post_valid(post_row) -> tuple[bool, str]:
  if post_row['selftext'].startswith('#'):
    return False, f"Post is a bot: {post_row['selftext']}"
  if post_row['num_comments'] == 0:
    return False, "Post has no comments"
  return True, ""
  

In [317]:
prompts: AlpacaPrompt = []

for i, row in df.iterrows():
  # check if post is valid
  valid, reason = is_post_valid(row)
  if not valid:
    logger.error(f"Post {row['name']} is not valid: {reason}, skipping")
    continue

  # post is valid but if it has no selftext, use title instead
  if row['selftext'] == '':
    logger.warning(f"Post {row['name']} has no selftext, using title instead")
    if row['title'] == '':
      logger.error(f"Post {row['name']} has no title, skipping")
      continue
    row['selftext'] = row['title']

  post_row = row
  # go down the comment chain in the post
  for comments in post_row['comments']:
    valid, reason = is_comment_valid(comments['body'])
    if not valid:
      logger.error(f"Comment {comments['name']} is not valid: {reason}, skipping")
      continue
    if comments['body'] == '':
      logger.warning(f"Comment {comments['name']} has no body, skipping")
      continue
    # input is the post body, response is the comment body
    prompts.append(AlpacaPrompt(
      instruction=instruction_types[0]['instruction'],
      input=post_row['selftext'],
      response=comments['body'],
    ))

    if comments['replies'] == []:
      logger.info(f"Comment {comments['name']} has no replies, end of chain")
      continue

    # go down the reply chain in the comment
    for reply in comments['replies']:
      if reply['body'] == '':
        logger.warning(f"Reply {reply['name']} has no body, skipping")
        continue
      # input is the comment body, response is the reply body
      prompts.append(AlpacaPrompt(
        instruction=instruction_types[1]['instruction'],
        input=comments['body'],
        response=reply['body'],
      ))

2024-11-24 22:50:22.089 | WARNING  | __main__:<module>:12 - Post t3_1dx1b0z has no selftext, using title instead
2024-11-24 22:50:22.090 | INFO     | __main__:<module>:36 - Comment t1_lbyv8mn has no replies, end of chain
2024-11-24 22:50:22.091 | INFO     | __main__:<module>:36 - Comment t1_lbyw648 has no replies, end of chain
2024-11-24 22:50:22.092 | INFO     | __main__:<module>:36 - Comment t1_lbycefd has no replies, end of chain
2024-11-24 22:50:22.092 | INFO     | __main__:<module>:36 - Comment t1_lbypxa3 has no replies, end of chain
2024-11-24 22:50:22.093 | INFO     | __main__:<module>:36 - Comment t1_lbz6gxk has no replies, end of chain
2024-11-24 22:50:22.093 | WARNING  | __main__:<module>:12 - Post t3_1dx15dz has no selftext, using title instead
2024-11-24 22:50:22.094 | INFO     | __main__:<module>:36 - Comment t1_lbyjcc6 has no replies, end of chain
2024-11-24 22:50:22.094 | INFO     | __main__:<module>:36 - Comment t1_lbyk9e3 has no replies, end of chain
2024-11-24 22:50:2

In [318]:
# write prompts to json file for alpaca
with open(f'datasets/{SUBREDDIT}_prompts.json'.lower(), 'w') as f:
  json.dump([prompt.to_dict() for prompt in prompts], f)

## 4.0 - Huggingface
Push the dataset to HuggingFace

In [ ]:
# TODO: PUSH THE DATASET TO HUGGINGFACE